In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110660  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,1.886213e+07
2018-02-28,1.585122e+07
2018-03-31,1.987184e+07
2018-04-30,1.932091e+07
2018-05-31,1.979899e+07
2018-06-30,1.850136e+07
2018-07-31,1.944032e+07
2018-08-31,1.996785e+07
2018-09-30,1.893544e+07


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    1.886213e+07
 2018-02-28    1.585122e+07
 2018-03-31    1.987184e+07
 2018-04-30    1.932091e+07
 2018-05-31    1.979899e+07
 2018-06-30    1.850136e+07
 2018-07-31    1.944032e+07
 2018-08-31    1.996785e+07
 2018-09-30    1.893544e+07
 2018-10-31    1.954216e+07
 2018-11-30    1.905918e+07
 2018-12-31    1.877705e+07
 2019-01-31    1.871225e+07
 2019-02-28    1.673932e+07
 2019-03-31    2.029907e+07
 2019-04-30    1.953982e+07
 2019-05-31    1.999525e+07
 2019-06-30    1.880313e+07
 2019-07-31    1.944006e+07
 2019-08-31    1.949425e+07
 2019-09-30    1.839220e+07
 2019-10-31    1.961651e+07
 2019-11-30    1.970444e+07
 2019-12-31    1.985981e+07
 2020-01-31    1.904437e+07
 2020-02-29    1.598360e+07
 2020-03-31    1.609480e+07
 2020-04-30    1.544665e+07
 2020-05-31    1.672632e+07
 2020-06-30    1.776197e+07
 2020-07-31    1.901462e+07
 2020-08-31    1.839075e+07
 2020-09-30    1.694485e+07
 2020-10-31    1.861633e+07
 2020-11-30    1.946759e+07

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.995791
p-value : 0.001433
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1309.364, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1305.110, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1303.799, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1303.593, Time=0.05 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1301.318, Time=0.16 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.31 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1303.404, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1306.013, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1307.665, Time=0.08 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1306.872, Time=0.14 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.40 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1305.499, Time=0.11 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1308.063, Time=0.17 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1309.778, Time=0.22 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1305.319, Time=0.30 sec
 ARIMA(3,0,0)(0,0,0)

ARIMA(order=(0, 0, 4), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(0,0,4))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 4)   Log Likelihood                -644.393
Method:                       css-mle   S.D. of innovations        1045441.895
Date:                Sat, 11 Sep 2021   AIC                           1300.787
Time:                        02:11:30   BIC                           1311.213
Sample:                             0   HQIC                          1304.608
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.844e+07   3.26e+05     56.555      0.000    1.78e+07    1.91e+07
ma.L1.y        0.5794      0.164      3.536      0.000       0.258       0.901
ma.L2.y        0.4523      0.179      2.527      0.0

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([18471093.78330529, 18441034.25730952]), array([1045441.89536737, 1208224.64221873]), array([[16422065.32045595, 20520122.24615463],
       [16072957.47332702, 20809111.04129203]]))
